In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/04 22:46:17 WARN Utils: Your hostname, asaad resolves to a loopback address: 127.0.1.1; using 192.168.1.26 instead (on interface wlp61s0)
24/03/04 22:46:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/04 22:46:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/03/04 22:46:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/03/04 22:46:29 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/03/04 22:46:29 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [3]:
pyspark.__version__

'3.3.2'

In [4]:
import pandas as pd

In [5]:
df_pandas = pd.read_csv('fhv_tripdata_2019-10.csv.gz')

In [35]:
    df_pandas.dtypes

dispatching_base_num       object
pickup_datetime            object
dropOff_datetime           object
PUlocationID              float64
DOlocationID              float64
SR_Flag                   float64
Affiliated_base_number     object
dtype: object

In [12]:
from pyspark.sql import types


In [64]:
# Define the schema with correct column names
schema = types.StructType([
    types.StructField('Affiliated_base_number', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropOff_datetime', types.TimestampType(), True),  # Corrected column name
    types.StructField('PUlocationID', types.IntegerType(), True),  # Corrected column name
    types.StructField('DOlocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.DoubleType(), True)  # Assuming SR_Flag is a numeric field
])

In [65]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhv_tripdata_2019-10.csv.gz')

In [66]:
df = df.repartition(6)


In [67]:
df.write.parquet('fhvhv/2019/10/')

24/03/04 23:48:01 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: dispatching_base_num, pickup_datetime, dropOff_datetime, PUlocationID, DOlocationID, SR_Flag, Affiliated_base_number
 Schema: Affiliated_base_number, dispatching_base_num, pickup_datetime, dropOff_datetime, PUlocationID, DOlocationID, SR_Flag
Expected: Affiliated_base_number but found: dispatching_base_num
CSV file: file:///home/asaasd/spark/homework/fhv_tripdata_2019-10.csv.gz


In [58]:
df = spark.read.parquet('fhvhv/2019/10/')

In [59]:
df.printSchema()


root
 |-- Affiliated_base_number: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [51]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col


In [50]:
df.show()

+----------------------+--------------------+-------------------+----------------+------------+------------+-------+
|Affiliated_base_number|dispatching_base_num|    pickup_datetime|dropOff_datetime|PULocationID|DOLocationID|SR_Flag|
+----------------------+--------------------+-------------------+----------------+------------+------------+-------+
|                B00837| 2019-10-03 09:48:29|2019-10-03 11:21:47|            null|         264|        null| B00837|
|                B00009| 2019-10-30 09:26:00|2019-10-30 09:43:00|            null|         264|        null| B00009|
|                B02594| 2019-10-04 19:27:00|2019-10-04 20:06:00|            null|         132|        null| B02594|
|                B00647| 2019-10-24 13:25:09|2019-10-24 15:40:36|            null|         241|        null| B00647|
|                B00900| 2019-10-14 06:17:14|2019-10-14 06:21:16|            null|         197|        null| B00900|
|                B02719| 2019-10-10 07:10:40|2019-10-10 07:25:31

In [63]:
not_null_rows = df.filter(col("DOLocationID").isNotNull()).show()


+----------------------+--------------------+---------------+----------------+------------+------------+-------+
|Affiliated_base_number|dispatching_base_num|pickup_datetime|dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+----------------------+--------------------+---------------+----------------+------------+------------+-------+
+----------------------+--------------------+---------------+----------------+------------+------------+-------+



In [26]:
october_15_trips = df.filter(df['dispatching_base_num'].cast("date") == "2019-10-15")

In [27]:
# Count the number of trips
number_of_trips = october_15_trips.count()

In [28]:
number_of_trips

62610